In [ ]:
import multiprocessing
import uuid
from itertools import repeat
from multiprocessing import Pool
from pathlib import Path

import pandas as pd
from sklearn.cluster import KMeans

import clustering_utils as utils
import pcap_generic_parser_helper as parser

# How to Run
1. Create your project folder in /data/input
2. Place your pcap and json files within your project folder
3. **Set the project_name variable below**
4. Run the Notebook from the start
5. Check the parser and clustering output in the csv files:
    - Parser all protocols &emsp;./data/output/\<project_name\>/all_parser_output.csv
    - Parser sip only &emsp;&emsp;&emsp;./data/output/\<project_name\>/sip_parser_output.csv
    - Clustering &emsp;&emsp;&emsp;&emsp;&emsp;./data/output/\<project_name\>/clustered_data.csv
6. Check Notebook **clustering_analysis.ipynb** for further insights into clustering results

In [ ]:
project_name = 'example_project'

In [ ]:
input_folder_path = Path.cwd().parent / 'data/input' / project_name
output_folder_path = Path.cwd().parent / 'data/output' / project_name
json_folder_path = output_folder_path / f'jsonfiles_{uuid.uuid4()}'

json_folder_path.mkdir(parents=True, exist_ok=True)

sip_output_filename = 'sip_parser_output.csv'
gtp_output_filename = 'gtp_parser_output.csv'
diameter_output_filename = 'diameter_parser_output.csv'
gtp_sip_output_csv_filename = 'gtp_sip_parser_output.csv'
all_output_csv_filename = 'all_parser_output.csv'
clustering_csv_filename = 'clustered_data.csv'

num_processors = multiprocessing.cpu_count()
p = Pool(processes=num_processors)

In [ ]:
# Get pcap files from ./data/input/<project_name>
pcap_files = [x for x in input_folder_path.glob('*.pcap')]
len(pcap_files)

In [ ]:
# Convert pcap files to json
result = p.starmap(parser.convert_pcap, zip(repeat(json_folder_path), pcap_files))

In [ ]:
# Get json files from ./data/input/<project_name> and converted ones 
json_files = [x for x in input_folder_path.glob('*.json')]
json_files = json_files + [x for x in json_folder_path.glob('*.json')]
len(json_files)

# Sip Parser

In [ ]:
# Read and parse json packets, generates a nest List
sip_parsed = p.map(parser.read_parse_sip, json_files)

In [ ]:
# Get output from original parser and concatenate both
sip_df_out_final = pd.DataFrame(data=sip_parsed)
sip_df_out_final = sip_df_out_final.fillna("*").set_index(0).rename_axis('pcap').add_prefix('sip ')

In [ ]:
sip_df_out_final.to_csv(output_folder_path / sip_output_filename)
sip_df_out_final

# GTPv2 Parser

In [ ]:
# Get all possible gtpv2 causes
gtp_list_from_tshark = p.map(parser.tshark_aggregate_gtp_cause, pcap_files)
gtp_causes = list({val for sublist in gtp_list_from_tshark for val in sublist if val != ''})

In [ ]:
# Read and parse json packets, generates a nest List
gtp_parse_output = p.starmap(parser.read_parse_gtp, zip(json_files, repeat(gtp_causes)))

In [ ]:
# Get output from original parser and concatenate both
gtp_df_out_final = pd.DataFrame(data=gtp_parse_output)
new_columns = ['pcap', 'n Requests', 'n Responses', 'n unanswered requests'] + ['cause = ' + str(cause) for cause in gtp_causes]
gtp_df_out_final = gtp_df_out_final.fillna("*").rename(columns={i: new_columns[i] for i in range(len(new_columns))}) \
    .set_index('pcap').add_prefix('gtpv2 ')

In [ ]:
gtp_df_out_final.to_csv(output_folder_path / gtp_output_filename)
gtp_df_out_final

# Diameter Parser

In [ ]:
# Get all possible diameter result codes
diameter_list_from_tshark = p.map(parser.tshark_aggregate_diameter_result_code, pcap_files)
diameter_result_codes = list({val for sublist in diameter_list_from_tshark for val in sublist if val != ''})

In [ ]:
# Read and parse json packets, generates a nest List
diameter_parse_output = p.starmap(parser.read_parse_diameter, zip(json_files, repeat(diameter_result_codes)))

In [ ]:
# Get output from original parser and concatenate both
diameter_df_out_final = pd.DataFrame(data=diameter_parse_output)
new_columns = ['pcap', 'n Requests', 'n Responses', 'n unanswered requests'] + ['result code = ' + str(code) for code in diameter_result_codes]
diameter_df_out_final = diameter_df_out_final.fillna("*").rename(columns={i: new_columns[i] for i in range(len(new_columns))}) \
    .set_index('pcap').add_prefix('diameter ')

In [ ]:
diameter_df_out_final.to_csv(output_folder_path / diameter_output_filename)
diameter_df_out_final

# Merge sip, gtpv2, and diameter Parser Outputs

In [ ]:
sip_gtp_df = pd.concat([gtp_df_out_final, sip_df_out_final], axis=1)
sip_gtp_df.to_csv(output_folder_path / gtp_sip_output_csv_filename)

In [ ]:
all_parser_out_df = pd.concat([diameter_df_out_final, gtp_df_out_final, sip_df_out_final], axis=1)
all_parser_out_df.to_csv(output_folder_path / all_output_csv_filename)
all_parser_out_df

# Cluster Parser Output

In [ ]:
ignore_cols = []
static_cols = ('gtpv2 n ', 'gtpv2 cause =', 'diameter n ', 'diameter result code =')
label = None
replace_none_values = '*'
use_encoder = True  # If True -> call one_hot_encoder
number_of_clusters = -1  # specify number of cluster. If -1 -> calculate optimal_cluster_num

In [ ]:
# drop duplicate pcaps
all_parser_out_df = all_parser_out_df[~all_parser_out_df.index.duplicated(keep='first')]
# drop columns with unique value
all_parser_out_df = all_parser_out_df[all_parser_out_df.columns[all_parser_out_df.nunique() > 1]]
# replace null values
all_parser_out_df = utils.transform_data(all_parser_out_df, replace_none_values)
all_parser_out_df

In [ ]:
clustering_data = all_parser_out_df[[x for x in all_parser_out_df.columns if x not in ignore_cols]]
if use_encoder:
    df_static = clustering_data[[x for x in clustering_data.columns if x.startswith(static_cols)]]
    df_static_set = set(df_static)
    df_dynamic = clustering_data[[x for x in clustering_data.columns if not x in df_static_set]]

    clustering_data = utils.one_hot_encoder(df_dynamic)
    clustering_data = pd.concat([df_static, clustering_data.set_index(df_static.index)], axis=1)
clustering_data

In [ ]:
if number_of_clusters == -1:
    number_of_clusters = utils.optimal_cluster_num(clustering_data)
params = {'n_clusters': number_of_clusters, 'init': 'k-means++', 'max_iter': 120, 'n_init': 25, 'random_state': 1}
clusterer = KMeans(**params)
clusters, silhouette = utils.cluster(clusterer, clustering_data)
print('number of clusters {}'.format(number_of_clusters))

In [ ]:
all_df, clusters_df, score, percent_mean, silhouette_mean = utils.score_fun(all_parser_out_df, clusters, silhouette, label)
print('Silhouette Mean {}'.format(silhouette_mean))
all_df.to_csv(output_folder_path / clustering_csv_filename)
all_df

In [ ]:
clusters_df